# Differentiable Subset Sum Problem

The **subset sum problem** is a fundamental NP-complete problem in computational complexity theory: given a set of integers and a target sum, determine if there exists a subset whose elements sum exactly to the target value. While this problem is computationally intractable for large instances using classical algorithms, we can approach it using differentiable programming techniques.

## Problem Definition

The [subset sum problem](https://en.wikipedia.org/wiki/Subset_sum_problem) is formally defined as follows: given a set of numbers $S = \{s_1, s_2, ..., s_n\}$ and a target value $T$, find a subset $S' \subseteq S$ such that $\sum_{s_i \in S'} s_i = T$.

In our differentiable approach, we reformulate this as finding a binary selection mask $M = [m_1, m_2, ..., m_n]$ where $m_i \in \{0, 1\}$ indicates whether element $s_i$ is included in the subset.

## Mathematical Formulation

Let's illustrate this with a concrete example. Given a set $S$ and a target $T$:

$$T = 7.0$$

$$S = \begin{bmatrix} 1.0 & 2.0 & 3.0 & 4.0 & 5.0 \end{bmatrix}$$

Our goal is to find a binary mask $M$ such that the dot product equals the target:

$$M \cdot S = T$$

Here's an example of a valid mask that achieves our target:

$$M = \begin{bmatrix} 0.0 & 0.0 & 1.0 & 1.0 & 0.0 \end{bmatrix}$$

We can verify: $0 \cdot 1 + 0 \cdot 2 + 1 \cdot 3 + 1 \cdot 4 + 0 \cdot 5 = 7$

## Differentiable Implementation

The key insight is to make the subset sum computation fully differentiable using TensorFlow operations. This enables gradient-based optimization to search for valid solutions, even though the underlying problem is discrete.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model, Input, layers

In [2]:
@tf.function
def compute_subset_sum(S, M):
    return tf.tensordot(S, M, 1)

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable([0,0,1,1,0],dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    T_ = compute_subset_sum(S, M)
    
print(T_)
print(tape.gradient(T_, S))
print(tape.gradient(T_, M))

tf.Tensor(7.0, shape=(), dtype=float32)
tf.Tensor([0. 0. 1. 1. 0.], shape=(5,), dtype=float32)
tf.Tensor([1. 2. 3. 4. 5.], shape=(5,), dtype=float32)


## Naive Training Approach

If we train directly using only the L2 loss between predicted and target sums, we encounter a fundamental problem: **the optimizer finds continuous solutions rather than binary ones**. 

The mask $M$ converges to fractional values (around 0.467 for each element in this case) rather than the desired binary values $\{0, 1\}$. This happens because the loss function only cares about achieving the target sum, not about maintaining the discrete nature of the selection.

### The Problem with Continuous Relaxation

When we relax the discrete constraint $m_i \in \{0, 1\}$ to $m_i \in [0, 1]$, the optimizer finds the "easy" solution of using fractional coefficients. While this achieves the target sum mathematically, it doesn't represent a valid subset selection.

## Training

However, if we train as is, we find that $M$ is not a mask but it forms a linear combination with its inputs.

## Enforcing Binary Solutions with Bistable Loss

To force the mask values toward binary solutions, we introduce the **bistable loss function**. This loss function penalizes values that are neither close to 0 nor close to 1, creating an energy landscape that favors binary solutions.

The bistable loss is defined as:
$$L_{bistable}(x) = 4x(1-x)$$

This function:
- **Minimizes** when $x = 0$ or $x = 1$ (binary values)
- **Maximizes** when $x = 0.5$ (maximally uncertain values)
- Creates a **double-well potential** that encourages binary convergence

By adding this regularization term to our primary loss, we guide the optimization toward discrete solutions while still allowing gradient-based training.

### Bistable loss
To force the values to be close to 0 and 1, we introduce the [Bistable Loss](notebooks/boolean-satisfiability.ipynb)

### Results with Bistable Loss

The bistable loss significantly improves convergence toward binary solutions. Observing the training output, we can see that:

1. **Initial iterations**: Values start near 1.0 and gradually differentiate
2. **Middle iterations**: Some values move toward 0, others toward 1
3. **Final iterations**: Values converge much closer to binary solutions

The combination of subset sum loss and bistable loss creates a more constrained optimization landscape that favors valid discrete solutions while maintaining differentiability for gradient-based training.

On retraining we find that each element the mask is now closer to 0 or 1

## One-Hot Softmax Approach

To further enforce binary constraints, we employ a **one-hot encoding with softmax** strategy. This approach transforms the problem into a more structured discrete choice.

### Dimensional Transformation

We expand the mask dimensionality from:
$$M = \begin{bmatrix} 0 & 0 & 1 & 1 & 0 \end{bmatrix}$$

to a 2D representation:
$$M = \begin{bmatrix} 1 & 1 & 0 & 0 & 1 \\ 0 & 0 & 1 & 1 & 0 \end{bmatrix}$$

where each column represents a binary choice between "exclude" (first row) and "include" (second row).

### Softmax Normalization

By applying softmax along the vertical axis, we ensure that each element has exactly one active choice:
$$M_s = \text{softmax}(M, \text{axis}=\text{vertical})$$

### Modified Computation

The subset sum computation becomes:
$$\bar{T} = \frac{(M_s[1] \cdot S) + ((1 - M_s[0]) \cdot S)}{2}$$

This formulation enforces mutual exclusivity between inclusion and exclusion decisions while maintaining differentiability.

### One hot softmax

To further make sure that the mask remains either 0 or 1, we increase the dimentionality of the $M$ and apply softmax along the vertical axis.

\begin{equation*}
M = 
\begin{bmatrix}
0 & 0 & 1 & 1 & 0 \\
\end{bmatrix}\end{equation*}

becomes

\begin{equation*}
M = 
\begin{bmatrix}
1 & 1 & 0 & 0 & 1 \\
0 & 0 & 1 & 1 & 0 \\
\end{bmatrix}\end{equation*}

Therefore, $\bar{T}$ becomes

\begin{equation*} 
M_s = softmax(M, axis=vertical)
\end{equation*}

\begin{equation*} 
\bar{T} = \frac{(M_s[1] \cdot S) + ((1 - M_s[0]) \cdot S)}{2} 
\end{equation*}

In [6]:
@tf.function
def compute_subset_sum_v2(S, M):
    M = tf.transpose(M)
    pos = tf.tensordot(S, M[1], 1)
    neg = tf.tensordot(S, 1 - M[0], 1)
    return (pos + neg) / 2

S = tf.Variable([1,2,3,4,5],dtype=tf.float32)
M = tf.Variable(tf.one_hot([0,0,1,1,0], 2),dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    M_s = tf.nn.softmax(M, axis=1)
    T_ = compute_subset_sum_v2(S, M_s)

tf.print(tf.transpose(M))
tf.print(T_)
tf.print(tape.gradient(T_, S))
tf.print(tape.gradient(T_, M))

[[1 1 0 0 1]
 [0 0 1 1 0]]
7.2689414
[0.268941402 0.268941402 0.731058598 0.731058598 0.268941402]
[[-0.196611926 0.196611941]
 [-0.393223852 0.393223882]
 [-0.589835823 0.589835823]
 [-0.786447763 0.786447704]
 [-0.983059645 0.983059704]]


### Comparative Performance

The combination of **bistable loss with one-hot softmax encoding** demonstrates superior performance compared to the naive approach:

1. **Faster Convergence**: The structured choice representation accelerates training
2. **Better Binary Solutions**: Values converge more reliably to near-binary states
3. **Stability**: The softmax constraint prevents gradient explosion and provides more stable training dynamics

The one-hot approach transforms the continuous relaxation problem into a structured discrete choice problem while maintaining end-to-end differentiability.

## Neural Network-Driven Candidate Generation

In the previous experiments, we manually initialized the mask $M$. Now we delegate this **candidate generation** to a neural network, creating an end-to-end learnable system.

### Architecture Overview

Let $G$ be our **candidate generator network**. The network takes a set $S$ and target $T$ as input and produces an initial guess $\bar{M}$:

$$\bar{M} = G(S, T)$$

### End-to-End Differentiable Pipeline

The complete system becomes:
$$\bar{T} = H(S, G(S, T))$$

where:
- $G$: Neural network that generates subset selection candidates
- $H$: Differentiable subset sum computation function
- $S$: Input set
- $T$: Target sum
- $\bar{T}$: Predicted sum

### Training Strategy

The neural network learns to generate good initial candidates by optimizing:
1. **Subset Sum Loss**: $|\bar{T} - T|$ to match the target
2. **Bistable Loss**: To encourage binary selections in the generated mask

This approach combines the power of **neural pattern recognition** (for generating candidates) with **differentiable algorithms** (for computing subset sums), creating a hybrid system that can tackle NP-complete problems through gradient-based optimization.

## Neural network driven candidate generation

In the last few experiments, the initial candidate $M$ was arbitarily chosen. Here, we shall now delegate this *guesswork* to a neural network.

Let $G$ be our DNN. The specifics of its architecture is not relevant as this is a toy problem. $G$ takes in an set $S$ and a target $T$ and tries to guess the correct answer $\bar{M}$ in one shot.

$$ \bar{M} = G(S, T) $$

A benefit of having the `compute_subset_sum` function (lets call it $H$) end-to-end differentiable is that it can now be interfaced seamlessly with DNNs. 

We get

\begin{equation*} 
\bar{T} = H(S, \bar{M})
\end{equation*}

\begin{equation*} 
or
\end{equation*}

\begin{equation*}
\bar{T} = H(S, G(S, T)) 
\end{equation*}

Now, we can use our bistable loss and $| \bar{T} - T | $ loss to train the system end-to-end.

### Candidate Generator Architecture

The `CandidateGeneratorBlock` is a simple feedforward network that:

1. **Input Processing**: Takes concatenated set and target `[S, T]` as input
2. **Feature Learning**: Uses two hidden layers with ReLU activation for feature extraction
3. **Output Generation**: Produces logits for binary choices and applies softmax
4. **Shape Transformation**: Reshapes output to `(batch_size, set_length, 2)` for one-hot encoding

The network learns to map from `(set + target)` pairs to binary selection patterns, effectively learning heuristics for the subset sum problem.

In [8]:
class CandidateGeneratorBlock(layers.Layer):
    def __init__(self, set_length):
        super(CandidateGeneratorBlock, self).__init__()
        self.set_length = set_length
        
        self.dense1 = layers.Dense(10, kernel_initializer="he_normal", activation='relu')
        self.dense2 = layers.Dense(10, kernel_initializer="he_normal", activation='relu')
        self.dense3 = layers.Dense(set_length * 2)

    def call(self, x):
        h = x
        h = self.dense1(h)
        h = self.dense2(h)
        h = self.dense3(h)
        h = tf.reshape(h, (-1, self.set_length, 2))
        h = tf.nn.softmax(h, axis=-1)
        return h

In [9]:
batch_size = 1
set_length = 5
temp_generator = CandidateGeneratorBlock(set_length)
input_shape = (batch_size, set_length + 1)
output_shape = (batch_size, set_length, 2)
x = tf.random.normal(input_shape)
y = tf.math.round(tf.random.uniform(output_shape, minval=0, maxval=1))
result = temp_generator(x)
print(x.shape, result.shape, y.shape)
print(result)

(1, 6) (1, 5, 2) (1, 5, 2)
tf.Tensor(
[[[0.7139912  0.28600878]
  [0.47499245 0.5250076 ]
  [0.4091633  0.59083676]
  [0.27171898 0.7282811 ]
  [0.57239777 0.42760226]]], shape=(1, 5, 2), dtype=float32)


### Batch Processing Implementation

The `compute_subset_sum_batch` function extends our differentiable computation to handle multiple examples simultaneously:

- **Tensor Operations**: Uses broadcasting for efficient batch computation
- **Gradient Flow**: Maintains full differentiability across the batch dimension
- **Memory Efficiency**: Processes multiple subset sum problems in parallel

This batched implementation is essential for training neural networks on large datasets of subset sum instances.

In [10]:
# batch_size = 10
# set_length = 5

# input_shape = (batch_size, set_length)
# output_shape = (batch_size, set_length, 2)
# x = tf.random.normal(input_shape)
# y = tf.math.round(tf.random.uniform(output_shape, minval=0, maxval=1))

# temp_generator = CandidateGeneratorBlock(set_length)
# a = Input(shape=(set_length))
# b = temp_generator(a)
# m = Model(inputs=a, outputs=b)
# m.compile(loss='mse', optimizer='adam')
# m.fit(x=x,y=y,epochs=100,batch_size=batch_size)

### Training Dataset Generation

We create a synthetic dataset of subset sum problems:

1. **Random Sets**: Generate random integer sets of fixed length
2. **Valid Solutions**: Create random binary masks for each set
3. **Target Computation**: Calculate the corresponding target sums
4. **Data Format**: Concatenate sets and targets for network input

This synthetic approach allows us to train on a large variety of subset sum instances with known ground truth solutions.

In [11]:
@tf.function
def compute_subset_sum_batch(S, M):
    M = tf.transpose(M,  [0, 2, 1])
    pos = tf.reduce_sum(S * M[:,1], axis=-1)
    neg = tf.reduce_sum(S * (1 - M[:,0]), axis=-1)
    return (pos + neg) / 2

dataset_size = 2
set_length = 5
h_S = np.random.randint(0, 10, (dataset_size, set_length))
h_M = np.random.randint(0, 2, (dataset_size, set_length))

S = tf.Variable(h_S, dtype=tf.float32)
M = tf.Variable(tf.one_hot(h_M, 2), dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    M_s = tf.nn.softmax(M, axis=-1)
    T_ = compute_subset_sum_batch(S, M_s)

tf.print(tf.transpose(M, [0, 2, 1]))
tf.print(T_)
tf.print(tape.gradient(T_, S))
tf.print(tape.gradient(T_, M))

[[[1 0 1 0 1]
  [0 1 0 1 0]]

 [[0 0 1 1 1]
  [1 1 0 0 0]]]
[16 4.30306244]
[[0.268941402 0.731058598 0.268941402 0.731058598 0.268941402]
 [0.731058598 0.731058598 0.268941402 0.268941402 0.268941402]]
[[[-1.76950729 1.76950753]
  [-1.37628365 1.37628353]
  [-0.786447704 0.786447763]
  [-1.76950753 1.76950729]
  [-0.589835823 0.589835823]]

 [[0 0]
  [0 0]
  [-1.37628353 1.37628365]
  [-0.786447704 0.786447763]
  [-0.983059645 0.983059704]]]


### Training Results Analysis

The training output shows the neural network learning to generate increasingly accurate subset selections:

1. **Early Training**: Network produces suboptimal candidates with high loss
2. **Mid Training**: Gradual improvement in candidate quality and target matching
3. **Late Training**: Convergence toward valid binary solutions with low error

The network learns to recognize patterns in subset sum problems and generate candidates that are close to optimal solutions, significantly reducing the search space for subsequent fine-tuning.

## Fine-Tuning Neural Network Candidates

Once the neural network generates an initial candidate $\bar{M}$, we can **fine-tune** this candidate using our differentiable subset sum function. This two-stage approach combines the strengths of both neural networks and optimization:

### Two-Stage Optimization Strategy

1. **Stage 1 - Neural Generation**: Use the trained network to quickly generate a reasonable initial candidate
2. **Stage 2 - Local Optimization**: Fine-tune the candidate using gradient descent with our subset sum and bistable losses

### Benefits of This Approach

- **Warm Start**: The neural network provides a much better initialization than random guessing
- **Faster Convergence**: Starting near a good solution reduces optimization time
- **Higher Success Rate**: Better initial candidates increase the likelihood of finding valid solutions
- **Scalability**: The neural network can generalize to new problem instances

### Implementation Details

The fine-tuning process uses the same loss combination as before:
- **Primary Loss**: L2 distance between predicted and target sums
- **Regularization**: Bistable loss to maintain binary constraints
- **Optimization**: Adam optimizer for stable gradient updates

This hybrid approach demonstrates how neural networks can be integrated with differentiable algorithms to tackle computationally hard problems.

In [13]:
dataset_size = 1000
set_length = 5
S = np.random.randint(0, 10, (dataset_size, set_length))
M = np.random.randint(0, 2, (dataset_size, set_length))
T = np.reshape(np.sum(S * M, axis=1), (dataset_size, 1))
ST = np.concatenate((S, T), axis=1)

S = np.float32(S)
T = np.float32(T)
ST = np.float32(ST)

## Summary and Key Insights

This notebook demonstrates a **differentiable programming approach** to the NP-complete subset sum problem, showcasing several important techniques:

### Technical Contributions

1. **Differentiable Formulation**: Reformulating discrete optimization as continuous optimization with gradient-based methods
2. **Bistable Loss Function**: Using specialized loss functions to encourage binary solutions while maintaining differentiability
3. **One-Hot Softmax Encoding**: Structured representation that enforces mutual exclusivity constraints
4. **Neural-Algorithmic Hybrid**: Combining neural networks for candidate generation with optimization-based fine-tuning

### Key Insights

- **Relaxation Strategy**: Continuous relaxation enables gradient-based optimization of discrete problems
- **Regularization Importance**: Bistable loss is crucial for recovering binary solutions from continuous optimization
- **Hybrid Approaches**: Neural networks excel at generating good initializations for traditional optimization
- **End-to-End Learning**: The entire pipeline remains differentiable, enabling sophisticated training strategies

### Applications and Extensions

This approach can be extended to other combinatorial optimization problems such as:
- **Graph Coloring**: Using similar binary selection mechanisms
- **Traveling Salesman Problem**: With appropriate sequence-based neural architectures
- **Boolean Satisfiability**: Extending the boolean logic framework from earlier notebooks
- **Resource Allocation**: In scheduling and planning domains

The techniques demonstrated here represent a general framework for making discrete optimization problems amenable to modern machine learning approaches while maintaining the expressiveness of classical algorithmic methods.

In [14]:
train_step(S[:2], T[:2], ST[:2])

(<tf.Tensor: shape=(), dtype=float32, numpy=36.61646>,
 <tf.Tensor: shape=(2, 5, 2), dtype=float32, numpy=
 array([[[9.99797642e-01, 2.02336319e-04],
         [1.36642475e-05, 9.99986291e-01],
         [5.92471451e-07, 9.99999404e-01],
         [7.41534710e-01, 2.58465230e-01],
         [9.99999762e-01, 2.91080596e-07]],
 
        [[9.69633400e-01, 3.03665511e-02],
         [2.41654180e-02, 9.75834608e-01],
         [2.50900686e-01, 7.49099314e-01],
         [9.10448074e-01, 8.95519182e-02],
         [6.18431568e-01, 3.81568432e-01]]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([14.068474,  8.259684], dtype=float32)>)

In [15]:
s = tf.constant(S[0], dtype=tf.float32)
s = tf.expand_dims(s, 0)

for i in range(1000):
    loss, m, t_ = train_step(S, T, ST)
    if i % 100 == 0:
        m = m[0]
        t_ = t_[0]
        m_t = tf.transpose(m)
        r_m = tf.round(m)
        r_m = tf.expand_dims(r_m, 0)
        actual = compute_subset_sum_batch(s, r_m)[0]
        tf.print(loss, m_t[1], t_, actual, int(T[0,0]))
        
tf.print(m_t)

38.0997086 [0.000234175328 0.999985218 0.999999404 0.309308052 2.78606336e-07] 14.4753399 12 21
10.1025267 [0.181596816 0.999999046 0.999754846 0.947611749 7.96055222e-09] 20.3053169 20 21
6.48026657 [0.56387651 0.999999404 0.960552514 0.997461319 1.07887754e-09] 21.9196129 24 21
5.73290539 [0.756967843 0.999997735 0.944468737 0.999148846 2.01228881e-10] 22.5768032 24 21
5.04855633 [0.951817334 0.999989033 0.886824906 0.999789774 3.92590821e-11] 22.9001427 24 21
4.72130775 [0.991167903 0.999964237 0.827799082 0.9999578 9.12365409e-12] 22.5865822 24 21
4.49034357 [0.998867631 0.999913454 0.796109259 0.999991298 2.26091736e-12] 22.3639297 24 21
4.30261469 [0.999887347 0.999844432 0.768945515 0.999997735 6.6921631e-13] 22.1504745 24 21
4.15217876 [0.999986887 0.999717176 0.748130858 0.999998927 2.8707085e-13] 21.9838562 24 21
4.01752472 [0.99999845 0.999705493 0.727931 0.999999404 1.01167677e-13] 21.822258 24 21
[[1.5196631e-06 0.000294527214 0.272068977 5.36702942e-07 1]
 [0.99999845 0.9

In [16]:
nS = np.array([[1, 2, 3, 4, 5]], dtype=np.float32)
nT = np.array([[7]], dtype=np.float32)
nST = np.concatenate((nS, nT), axis=1)
nM = generator(nST)
np.round(nM)[0].T

array([[0., 0., 1., 0., 1.],
       [1., 1., 0., 1., 0.]], dtype=float32)

### Fine tuning the candidate

Once the neural network has guessed the initial candidate $\bar{M}$, we can now fine tune it using `compute_subset_sum` function. Assuming the neural network has learnt a good representation of the problem, the candidate should equal or at least be close to the solution. Therefore, we could expect it to converge faster and more reliably than manually guessing a candidate.

In [17]:
opt = tf.keras.optimizers.Adam()

@tf.function
def fine_tune_step(S, M, T):
    with tf.GradientTape() as tape:
        M_s = tf.nn.softmax(M, axis=1)
        T_ = compute_subset_sum_v2(S, M_s)
        loss = tf.nn.l2_loss(T_ - T)
        loss += tf.reduce_sum(bistable_loss(M_s)) * 10
    
    grads = tape.gradient(loss, M)
    opt.apply_gradients(zip([grads], [M]))
    
    return loss, T_

S = tf.Variable(nS,dtype=tf.float32)
M = tf.Variable(nM[0], dtype=tf.float32)
T = 7

for i in range(1000):
    loss, T_ = fine_tune_step(S, M, T)
    if i % 100 == 0:
        M_T = tf.transpose(M)
        M_T = tf.nn.softmax(M_T, axis=0)
        M_s = tf.nn.softmax(M, axis=1)
        actual = compute_subset_sum_v2(S, tf.round(M_s))
        tf.print(loss, M_T[1], T_[0], actual[0])
        
tf.print(M_T)

4.25990248 [0.723222 0.650265336 0.291595429 0.729313731 0.268548429] 7.15885067 7
3.55748272 [0.76129365 0.69552058 0.252263904 0.766952276 0.231619418] 7.13519096 7
2.89820313 [0.794480681 0.738415241 0.217733368 0.799760461 0.199965134] 7.12346792 7
2.3320303 [0.822267354 0.775921702 0.188515246 0.82707417 0.173519135] 7.11562 7
1.87315917 [0.845076859 0.807170928 0.164326832 0.84936887 0.151737064] 7.10862875 7
1.51269031 [0.863694 0.832624137 0.144478917 0.867491841 0.133874401] 7.10178614 7
1.2330004 [0.878932178 0.853232145 0.128189027 0.882288456 0.119186744] 7.09511614 7
1.01610804 [0.89149785 0.869977891 0.114743605 0.894473612 0.107026093] 7.08877087 7
0.846875072 [0.901959956 0.883702695 0.103550903 0.904613614 0.0968662798] 7.08286 7
0.713556945 [0.910761774 0.895073354 0.0941422805 0.913143694 0.0882939249] 7.07743168 7
[[0.0892381892 0.104926601 0.905857742 0.0868563354 0.91170609]
 [0.910761774 0.895073354 0.0941422805 0.913143694 0.0882939249]]
